# **Facial Expression Recognition**

In [ ]:
# Libraries

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Input,Conv2D,MaxPooling2D,Flatten,Dropout,BatchNormalization,Dense
from keras.preprocessing.image import ImageDataGenerator as Imgen
from keras.callbacks import EarlyStopping,ModelCheckpoint

from sklearn.metrics import confusion_matrix,classification_report

## **Reading and Organiing Data**

In [ ]:
# reading the data
data = pd.read_csv("../input/challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv")
data.head()

In [ ]:
cols = ["emotion","subset","pixels"]
data.columns = cols
data.columns

emotions = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

In [ ]:
print(data.subset.value_counts())
print(data.emotion.value_counts())

In [ ]:
# Splitting into train test and validate

train_data = data[data["subset"]=="Training"]
val_data = data[data["subset"]=="PublicTest"]
test_data = data[data["subset"]=="PrivateTest"]

print(train_data.shape,test_data.shape,val_data.shape)

In [ ]:
# to extract image data from pixel column

def toPixels(pixels):

    arr = np.array(pixels.split(),"float64")
    return arr

def reshapetoImage(data):

    Images = np.reshape(data["pixels"].to_list(),(data.shape[0],48,48,1))

    return Images

In [ ]:
train_data["pixels"] = train_data["pixels"].apply(toPixels)
val_data["pixels"] = val_data["pixels"].apply(toPixels)
test_data["pixels"] = test_data["pixels"].apply(toPixels)

In [ ]:
# images and labels

train_images = reshapetoImage(train_data)
val_images = reshapetoImage(val_data)
test_images = reshapetoImage(test_data)

train_labels = train_data["emotion"]
val_labels = val_data["emotion"]
test_labels = test_data["emotion"]

### **Visualaizing some Images**

In [ ]:
def plotImages(x,y):
    plt.figure(figsize=[20,12])
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.imshow(x[i],cmap="gray")
        plt.title(emotions[y[i]])
        plt.axis("off")
    plt.show()

In [ ]:
plotImages(train_images,train_labels)

## **Image Data Generator**

In [ ]:
trainGen = Imgen(rescale=1./255,
                 zoom_range=0.2,
                 shear_range=0.2,
                 horizontal_flip=True
                 )
valGen = Imgen(rescale=1./255,
                 zoom_range=0.2,
                 shear_range=0.2,
                 horizontal_flip=True
               )
testGen = Imgen(rescale=1./255)

In [ ]:
trainds = trainGen.flow(train_images,train_labels,
                   batch_size = 32
                   )

valds = valGen.flow(val_images,val_labels,
               batch_size = 32
               )

testds = testGen.flow(test_images,test_labels,
                      batch_size=32,
                      shuffle=False)

**One batch**

In [ ]:
# one batch
x,y = next(trainds)

# see 
plotImages(x,y)

## **Model**

In [ ]:
# Model
image_input = Input(shape=(48,48,1))

x = Conv2D(64,(3,3), activation="relu")(image_input)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.3)(x)

x = Conv2D(128,(3,3),activation="relu")(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.3)(x)

x = Conv2D(512,(3,3),activation="relu")(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.3)(x)

x = Conv2D(512,(3,3),activation="relu")(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.3)(x)

x = Flatten()(x)

x = Dense(512,activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(256,activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

image_output = Dense(7,activation="softmax")(x)

model = Model(image_input,image_output)

print(model.summary())

In [ ]:
# Compile the model
print("Compiling the Model....")
model.compile(optimizer='adam',loss = 'sparse_categorical_crossentropy', metrics=["accuracy"])
print("Model Compiled!")

In [ ]:
# Defining callbacks
my_calls = [EarlyStopping(monitor='val_accuracy',patience=5),
            ModelCheckpoint("Model_ethnicity.h5",verbose=1,save_best_only=True)]

**Train**

In [ ]:
hist = model.fit(trainds, epochs=40, validation_data=valds, callbacks=my_calls)

**Test**

In [ ]:
model.evaluate(testds)

**Graph**

In [ ]:
plt.figure(figsize=(15,6))

plt.subplot(1,2,1)
plt.plot(hist.epoch,hist.history['accuracy'],label = 'Training')
plt.plot(hist.epoch,hist.history['val_accuracy'],label = 'validation')

plt.title("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(hist.epoch,hist.history['loss'],label = 'Training')
plt.plot(hist.epoch,hist.history['val_loss'],label = 'validation')

plt.title("Loss")
plt.legend()
plt.show()


## **Predictions**

In [ ]:
pred = model.predict(testds)

In [ ]:
pred = [np.argmax(i) for i in pred]

In [ ]:
y_test = np.array(test_labels)

In [ ]:
print(classification_report(y_test,pred))

In [ ]:
plt.figure(figsize=[20,14])
sns.heatmap(confusion_matrix(y_test,pred),annot = True, fmt= "d", cmap = "Blues");